# Semi-Supervised Classification with Graph Convolutional Network Implementation

## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
!wget https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz
!tar -xvzf cora.tgz
!rm -r cora.tgz


--2024-07-22 09:27:35--  https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz
Resolving linqs-data.soe.ucsc.edu (linqs-data.soe.ucsc.edu)... 128.114.47.74
Connecting to linqs-data.soe.ucsc.edu (linqs-data.soe.ucsc.edu)|128.114.47.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168052 (164K) [application/x-gzip]
Saving to: ‘cora.tgz’

cora.tgz            100%[===================>] 164.11K   154KB/s    in 1.1s    

2024-07-22 09:27:41 (154 KB/s) - ‘cora.tgz’ saved [168052/168052]

cora/
cora/README
cora/cora.cites
cora/cora.content


In [2]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root = 'Citeseer', name = 'Citeseer')

In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader

# Step 1: Load the Citeseer Dataset
dataset = Planetoid(root = 'Citeseer', name = 'Citeseer')
# Step 2: Define the GCN Model
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize model, optimizer, and other utilities
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(input_dim=dataset.num_node_features, hidden_dim=16, output_dim=dataset.num_classes).to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Step 3: Training Loop
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

# Step 4: Test Function
def test():
    model.eval()
    logits, accs = model(data), []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

# Training and Evaluation
for epoch in range(200):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')


Epoch: 000, Loss: 1.7864, Train Acc: 0.8917, Val Acc: 0.4940, Test Acc: 0.5070
Epoch: 010, Loss: 0.1647, Train Acc: 1.0000, Val Acc: 0.6800, Test Acc: 0.6680
Epoch: 020, Loss: 0.0193, Train Acc: 1.0000, Val Acc: 0.6720, Test Acc: 0.6650
Epoch: 030, Loss: 0.0072, Train Acc: 1.0000, Val Acc: 0.6760, Test Acc: 0.6580
Epoch: 040, Loss: 0.0067, Train Acc: 1.0000, Val Acc: 0.6740, Test Acc: 0.6720
Epoch: 050, Loss: 0.0090, Train Acc: 1.0000, Val Acc: 0.6820, Test Acc: 0.6790
Epoch: 060, Loss: 0.0113, Train Acc: 1.0000, Val Acc: 0.6840, Test Acc: 0.6760
Epoch: 070, Loss: 0.0121, Train Acc: 1.0000, Val Acc: 0.6920, Test Acc: 0.6800
Epoch: 080, Loss: 0.0116, Train Acc: 1.0000, Val Acc: 0.6920, Test Acc: 0.6790
Epoch: 090, Loss: 0.0108, Train Acc: 1.0000, Val Acc: 0.6880, Test Acc: 0.6820
Epoch: 100, Loss: 0.0103, Train Acc: 1.0000, Val Acc: 0.6880, Test Acc: 0.6830
Epoch: 110, Loss: 0.0097, Train Acc: 1.0000, Val Acc: 0.6900, Test Acc: 0.6830
Epoch: 120, Loss: 0.0093, Train Acc: 1.0000, Val Acc

In [7]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import add_self_loops, degree

# Load the Citeseer dataset
dataset = Planetoid(root = 'Citeseer', name = 'Citeseer')
data = dataset[0]

# Step 1: Precompute normalized adjacency matrix
def normalize_adjacency(edge_index, num_nodes):
    # Add self-loops to the adjacency matrix
    edge_index, _ = add_self_loops(edge_index, num_nodes=num_nodes)
    
    # Compute degree of each node
    row, col = edge_index
    deg = degree(row, num_nodes=num_nodes)
    
    # Compute D^(-1/2)
    deg_inv_sqrt = deg.pow(-0.5)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0  # Handle zero degree
    
    # Normalize adjacency matrix: D^(-1/2) A D^(-1/2)
    norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
    return edge_index, norm

edge_index, norm = normalize_adjacency(data.edge_index, data.num_nodes)

# Step 2: Define GCN model
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.W1 = torch.nn.Parameter(torch.randn(input_dim, hidden_dim))
        self.W2 = torch.nn.Parameter(torch.randn(hidden_dim, output_dim))
        self.relu = torch.nn.ReLU()

    def forward(self, x, edge_index, norm):
        # First layer: H = ReLU(D^(-1/2) A D^(-1/2) X W1)
        x = torch.mm(x, self.W1)
        row, col = edge_index
        x = torch.sparse.mm(torch.sparse_coo_tensor(edge_index, norm, (data.num_nodes, data.num_nodes)), x)
        x = self.relu(x)
        
        # Second layer: H = D^(-1/2) A D^(-1/2) H W2
        x = torch.mm(x, self.W2)
        x = torch.sparse.mm(torch.sparse_coo_tensor(edge_index, norm, (data.num_nodes, data.num_nodes)), x)
        return F.log_softmax(x, dim=1)

# Initialize model, optimizer, and data
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(input_dim=dataset.num_node_features, hidden_dim=16, output_dim=dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Training loop
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, edge_index, norm)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

# Testing loop
def test():
    model.eval()
    logits, accs = model(data.x, edge_index, norm), []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

# Training and evaluation
for epoch in range(200):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')


Epoch: 000, Loss: 11.0108, Train Acc: 0.1583, Val Acc: 0.1720, Test Acc: 0.1960
Epoch: 010, Loss: 1.7516, Train Acc: 0.7000, Val Acc: 0.2840, Test Acc: 0.3230
Epoch: 020, Loss: 0.2621, Train Acc: 0.9333, Val Acc: 0.3560, Test Acc: 0.4030
Epoch: 030, Loss: 0.0431, Train Acc: 1.0000, Val Acc: 0.3860, Test Acc: 0.4190
Epoch: 040, Loss: 0.0243, Train Acc: 1.0000, Val Acc: 0.3880, Test Acc: 0.4240
Epoch: 050, Loss: 0.0183, Train Acc: 1.0000, Val Acc: 0.3960, Test Acc: 0.4290
Epoch: 060, Loss: 0.0168, Train Acc: 1.0000, Val Acc: 0.3880, Test Acc: 0.4300
Epoch: 070, Loss: 0.0167, Train Acc: 1.0000, Val Acc: 0.3800, Test Acc: 0.4250
Epoch: 080, Loss: 0.0170, Train Acc: 1.0000, Val Acc: 0.3860, Test Acc: 0.4260
Epoch: 090, Loss: 0.0170, Train Acc: 1.0000, Val Acc: 0.3920, Test Acc: 0.4270
Epoch: 100, Loss: 0.0167, Train Acc: 1.0000, Val Acc: 0.3960, Test Acc: 0.4240
Epoch: 110, Loss: 0.0162, Train Acc: 1.0000, Val Acc: 0.3960, Test Acc: 0.4240
Epoch: 120, Loss: 0.0156, Train Acc: 1.0000, Val Ac

In [19]:
class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x, adj):
        x = self.linear(x)
        x = torch.mm(adj, x)
        return x

class GCN(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(GCN, self).__init__()
        self.conv1 = GCNLayer(in_features, hidden_features)
        self.conv2 = GCNLayer(hidden_features, out_features)

    def forward(self, x, adj):
        x = self.conv1(x, adj)
        x = F.relu(x)
        x = self.conv2(x, adj)
        return F.log_softmax(x, dim=1)

In [20]:
model = GCN(features.shape[1], 16, len(le.classes_))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def train():
    model.train()
    optimizer.zero_grad()
    output = model(features, adj_matrix)
    loss = F.nll_loss(output[train_mask], tensor_encoded_labels[train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    output = model(features, adj_matrix)
    preds = output.argmax(dim=1)
    train_correct = preds[train_mask] == tensor_encoded_labels[train_mask]
    val_correct = preds[val_mask] == tensor_encoded_labels[val_mask]
    test_correct = preds[test_mask] == tensor_encoded_labels[test_mask]
    
    train_acc = train_correct.sum().item() / train_mask.sum().item()
    val_acc = val_correct.sum().item() / val_mask.sum().item()
    test_acc = test_correct.sum().item() / test_mask.sum().item()
    
    return train_acc, val_acc, test_acc

In [21]:

for epoch in range(200):
    loss = train()
    train_acc, val_acc, test_acc = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 000, Loss: 1.9253, Train Acc: 0.4161, Val Acc: 0.4240, Test Acc: 0.4373
Epoch: 001, Loss: 1.8598, Train Acc: 0.4520, Val Acc: 0.4562, Test Acc: 0.4539
Epoch: 002, Loss: 1.7820, Train Acc: 0.4813, Val Acc: 0.4839, Test Acc: 0.4779
Epoch: 003, Loss: 1.6954, Train Acc: 0.5003, Val Acc: 0.4977, Test Acc: 0.4889
Epoch: 004, Loss: 1.6063, Train Acc: 0.5146, Val Acc: 0.5069, Test Acc: 0.5074
Epoch: 005, Loss: 1.5167, Train Acc: 0.5434, Val Acc: 0.5207, Test Acc: 0.5351
Epoch: 006, Loss: 1.4265, Train Acc: 0.5798, Val Acc: 0.5438, Test Acc: 0.5590
Epoch: 007, Loss: 1.3367, Train Acc: 0.6198, Val Acc: 0.5668, Test Acc: 0.6107
Epoch: 008, Loss: 1.2482, Train Acc: 0.6752, Val Acc: 0.6498, Test Acc: 0.6605
Epoch: 009, Loss: 1.1608, Train Acc: 0.7245, Val Acc: 0.6912, Test Acc: 0.6937
Epoch: 010, Loss: 1.0744, Train Acc: 0.7737, Val Acc: 0.7189, Test Acc: 0.7306
Epoch: 011, Loss: 0.9899, Train Acc: 0.8061, Val Acc: 0.7558, Test Acc: 0.7620
Epoch: 012, Loss: 0.9086, Train Acc: 0.8338, Val Acc